In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from collections import OrderedDict

import albumentations as A
import cv2
import numpy as np
import torch
import torch.nn.functional as F
import torchvision.models as models
import torchvision.models.detection as detection
from albumentations.pytorch import ToTensorV2
from matplotlib import pyplot as plt
from pytorch_lightning import Trainer
from torch import Tensor, nn
from torch.utils.data import DataLoader
from torchvision.ops import RoIAlign
from torchvision.transforms import ToPILImage

from anomalib.config import get_configurable_parameters
from anomalib.data import InferenceDataset, get_datamodule
from anomalib.models import get_model
from anomalib.models.rkde.feature import FeatureExtractor as FeatureExtractor1
from anomalib.models.rkde.feature_extractor import FeatureExtractor as FeatureExtractor2
from anomalib.models.rkde.feature_extractor import RegionExtractor as RegionExtractor2
from anomalib.models.rkde.region import RegionExtractor as RegionExtractor1
from anomalib.models.rkde.torch_model import RkdeModel
from anomalib.pre_processing.pre_process import PreProcessor, get_transforms
from anomalib.utils.callbacks import LoadModelCallback, get_callbacks
from anomalib.utils.loggers import configure_logger, get_experiment_logger

In [ ]:
config = get_configurable_parameters(config_path="./anomalib/models/rkde/ucsd.yaml")

datamodule = get_datamodule(config)
model = get_model(config)
experiment_logger = get_experiment_logger(config)
callbacks = get_callbacks(config)

trainer = Trainer(**config.trainer, logger=experiment_logger, callbacks=callbacks)
trainer.fit(model=model, datamodule=datamodule)
# TODO: Training returns an error!

In [ ]:
filename = "anomalib/models/rkde/150.tif"
image = cv2.imread(filename)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformations.
transforms = get_transforms(config="anomalib/models/rkde/transforms.yaml")
pre_process = PreProcessor(config=transforms)

# Get the data via dataloader
dataset = InferenceDataset(path="anomalib/models/rkde/", pre_process=pre_process)
dataloader = DataLoader(dataset, batch_size=2)
i, data = next(enumerate(dataloader))
# datamodule = get_datamodule(config=config)
# datamodule.setup()
# i, data = next(enumerate(datamodule.test_dataloader()))

# Create the region extractor.
stage="rcnn"
use_original = False
region_extractor1 = RegionExtractor1(stage=stage, use_original=use_original).eval().cuda()
region_extractor2 = RegionExtractor2(stage=stage, use_original=use_original).eval().cuda()
torch_model = RkdeModel(region_extractor_stage=stage).cuda()

# Forward-Pass the input
boxes1 = region_extractor1([image])
boxes2 = region_extractor2(data["image"].cuda())

# Feature Extractor
feature_extractor1 = FeatureExtractor1().eval().cuda()
feature_extractor2 = FeatureExtractor2().eval().cuda()
features1 = feature_extractor1(image, boxes1[0])
features2 = feature_extractor2(data["image"].cuda())
features3 = torch_model.get_features(data["image"].cuda())

In [ ]:
features3.shape

In [ ]:
boxes1[0].shape

In [ ]:
F.pad(input=boxes2, pad=(1, 0, 0, 0), mode="constant", value=0)

In [ ]:
torch.cat(boxes2, dim=0).shape

In [ ]:
boxes_pt = torch.tensor(boxes1[0], dtype=torch.float32)
rois = torch.cat((torch.zeros(boxes_pt.size(0), 1), boxes_pt), 1).unsqueeze(0)
boxes_pt.shape, rois.shape

In [ ]:
torch.cat((torch.zeros(boxes_pt.size(0), 1), boxes_pt), 1).shape, boxes_pt.shape

In [ ]:
rois.shape